In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

# 311 Data

In [2]:
df311_og = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv')
df311_og.tail()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
164488,51836466,09/10/2021 09:40:00 PM,09/11/2021 12:00:00 PM,DSNY,Department of Sanitation,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,11214.0,2172 86 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.602252,-73.994325,"(40.602251784945565, -73.99432455291887)"
164489,51836565,09/10/2021 06:06:00 PM,09/11/2021 12:00:00 PM,DSNY,Department of Sanitation,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,11231.0,146 COLUMBIA STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.686904,-74.001806,"(40.68690380398603, -74.00180647759399)"
164490,51836576,09/10/2021 11:03:00 PM,09/11/2021 12:00:00 PM,DSNY,Department of Sanitation,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,11229.0,2272 EAST 17 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.595274,-73.953813,"(40.59527360176615, -73.95381259192706)"
164491,51912087,09/17/2021 08:24:00 PM,09/20/2021 12:00:00 PM,DSNY,Department of Sanitation,Dirty Conditions,E5 Loose Rubbish,Sidewalk,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164492,51912497,09/17/2021 12:43:00 PM,09/22/2021 12:00:00 PM,DSNY,Department of Sanitation,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df311 = df311_og[['Created Date','Borough','Complaint Type','Descriptor','Location Type','Incident Zip','Community Board', 'Location']]

#### Clearning Community District Column for a Merge

In [4]:
df311['Borough'].replace({'BROOKLYN':'BK',"QUEENS":'QN','BRONX':'BX','MANHATTAN':'MN','STATEN ISLAND':'SI'},inplace=True)
df311['CD'] = df311.Borough + df311['Community Board'].str[0:2]
df311.drop(['Borough','Community Board'],axis=1,inplace=True)
df311.head()

,Created Date,Complaint Type,Descriptor,Location Type,Incident Zip,Location,CD
0,05/03/2021 06:54:00 AM,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,11216.0,"(40.68347186202379, -73.9539209649856)",BK03
1,05/02/2021 08:26:00 AM,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,11105.0,"(40.777907404785, -73.90745125248668)",QN01
2,05/03/2021 10:35:00 AM,Dirty Conditions,E1 Improper Disposal,Sidewalk,10452.0,"(40.83393144965909, -73.92882432434307)",BX04
3,05/02/2021 02:34:00 PM,Dirty Conditions,E3A Dirty Area/Alleyway,Lot,10011.0,"(40.73463243696263, -73.99644940675921)",MN02
4,05/02/2021 03:03:00 PM,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,10038.0,"(40.706866299284606, -74.00699002392852)",MN01


#### Temporal Dimension

In [5]:
import datetime

In [6]:
df311['Created Date']= pd.to_datetime(df311['Created Date'])

In [7]:
df311.head()

,Created Date,Complaint Type,Descriptor,Location Type,Incident Zip,Location,CD
0,2021-05-03 06:54:00,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,11216.0,"(40.68347186202379, -73.9539209649856)",BK03
1,2021-05-02 08:26:00,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,11105.0,"(40.777907404785, -73.90745125248668)",QN01
2,2021-05-03 10:35:00,Dirty Conditions,E1 Improper Disposal,Sidewalk,10452.0,"(40.83393144965909, -73.92882432434307)",BX04
3,2021-05-02 14:34:00,Dirty Conditions,E3A Dirty Area/Alleyway,Lot,10011.0,"(40.73463243696263, -73.99644940675921)",MN02
4,2021-05-02 15:03:00,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,10038.0,"(40.706866299284606, -74.00699002392852)",MN01


In [8]:
list_of_months = pd.date_range('2017-01-01','2021-12-31' , freq='1M')-pd.offsets.MonthBegin(1)
len(list_of_months)
max(list_of_months)

Timestamp('2021-12-01 00:00:00')

In [9]:
df311_final = pd.DataFrame(columns=df311.columns)
for month in list_of_months:
    df311['month'] = month
    df311_final = pd.concat([df311_final, df311])

df311_final.shape

(9869580, 8)

In [10]:
len(df311_final['month'].unique())
#makes sense because 12 months x 5 years (2017-2021)

60

In [11]:
df311_final['Month'] = df311_final['month'].dt.to_period('M')

In [12]:
df311_final['Month'] = df311_final['Month'].values.astype('datetime64[M]')

In [13]:
df311_final['Month']=pd.to_datetime(df311_final['Month'], format='%Y-%m')

In [14]:
df311_final.dtypes

Created Date      datetime64[ns]
Complaint Type            object
Descriptor                object
Location Type             object
Incident Zip             float64
Location                  object
CD                        object
month             datetime64[ns]
Month             datetime64[ns]
dtype: object

In [15]:
df311_final = df311_final[['Month','CD','Complaint Type','Descriptor','Location Type','Incident Zip','Location']]
df311_final.head()

,Month,CD,Complaint Type,Descriptor,Location Type,Incident Zip,Location
0,2017-01-01,BK03,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,11216.0,"(40.68347186202379, -73.9539209649856)"
1,2017-01-01,QN01,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,11105.0,"(40.777907404785, -73.90745125248668)"
2,2017-01-01,BX04,Dirty Conditions,E1 Improper Disposal,Sidewalk,10452.0,"(40.83393144965909, -73.92882432434307)"
3,2017-01-01,MN02,Dirty Conditions,E3A Dirty Area/Alleyway,Lot,10011.0,"(40.73463243696263, -73.99644940675921)"
4,2017-01-01,MN01,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,10038.0,"(40.706866299284606, -74.00699002392852)"


In [16]:
#df311_final.to_csv('311.csv')

# Rating Data 

In [17]:
rating_og = pd.read_csv('Scorecard_Ratings.csv')
rating_og.head()

,Month,Borough,Community Board,District,Cleaning Section,Acceptable Streets %,Acceptable Sidewalks %,Acceptable Streets % - Previous Month,Acceptable Sidewalks % - Previous Month,Acceptable Streets % - Previous Year,Acceptable Sidewalks % - Previous Year,Acceptable Streets % - Previous Fiscal Quarter,Acceptable Sidewalks % - Previous Fiscal Quarter
0,2020 / 08,Manhattan,1,MN01,MN011,100.00,100.0,NaN,NaN,96.15,100.0,NaN,NaN
1,2020 / 08,Manhattan,1,MN01,MN013,100.00,100.0,NaN,NaN,100.00,100.0,NaN,NaN
2,2020 / 08,Manhattan,10,MN10,MN101,88.00,100.0,76.00,92.0,92.00,96.0,NaN,NaN
3,2020 / 08,Manhattan,10,MN10,MN103,91.67,100.0,60.87,100.0,96.30,100.0,NaN,NaN
4,2020 / 08,Manhattan,10,MN10,MN104,96.00,96.0,56.00,96.0,100.00,100.0,NaN,NaN


In [20]:
rating_og['Month'].unique()

array(['2020 / 08', '2018 / 10', '2018 / 09', '2018 / 08', '2018 / 07',
       '2018 / 06', '2018 / 05', '2018 / 04', '2018 / 03', '2018 / 02',
       '2018 / 01', '2017 / 12', '2017 / 11', '2017 / 10', '2017 / 09',
       '2017 / 08', '2017 / 07', '2017 / 06', '2017 / 05', '2017 / 04',
       '2017 / 03', '2017 / 02', '2017 / 01', '2020 / 09', '2021 / 04',
       '2020 / 10', '2020 / 11', '2021 / 05', '2020 / 12', '2021 / 01',
       '2021 / 03', '2021 / 02', '2021 / 06', '2021 / 08', '2021 / 10',
       '2021 / 09', '2021 / 07'], dtype=object)

In [21]:
rating = rating_og.drop(['Borough','Community Board','Cleaning Section'],axis=1)

### Temporal Dimension

In [22]:
rating['Month']=pd.to_datetime(rating['Month'], format='%Y / %m')
rating

,Month,District,Acceptable Streets %,Acceptable Sidewalks %,Acceptable Streets % - Previous Month,Acceptable Sidewalks % - Previous Month,Acceptable Streets % - Previous Year,Acceptable Sidewalks % - Previous Year,Acceptable Streets % - Previous Fiscal Quarter,Acceptable Sidewalks % - Previous Fiscal Quarter
0,2020-08-01,MN01,100.00,100.0,NaN,NaN,96.15,100.0,NaN,NaN
1,2020-08-01,MN01,100.00,100.0,NaN,NaN,100.00,100.0,NaN,NaN
2,2020-08-01,MN10,88.00,100.0,76.00,92.0,92.00,96.0,NaN,NaN
3,2020-08-01,MN10,91.67,100.0,60.87,100.0,96.30,100.0,NaN,NaN
4,2020-08-01,MN10,96.00,96.0,56.00,96.0,100.00,100.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
8615,2021-07-01,SI02,100.00,100.0,100.00,100.0,100.00,100.0,100.0,100.0
8616,2021-07-01,SI03,100.00,100.0,100.00,100.0,100.00,100.0,100.0,100.0
8617,2021-07-01,SI03,100.00,100.0,100.00,100.0,100.00,100.0,100.0,100.0
8618,2021-07-01,SI03,100.00,100.0,100.00,100.0,100.00,100.0,100.0,100.0


In [24]:
rating = rating.sort_values(by='Month')
rating

,Month,District,Acceptable Streets %,Acceptable Sidewalks %,Acceptable Streets % - Previous Month,Acceptable Sidewalks % - Previous Month,Acceptable Streets % - Previous Year,Acceptable Sidewalks % - Previous Year,Acceptable Streets % - Previous Fiscal Quarter,Acceptable Sidewalks % - Previous Fiscal Quarter
5208,2017-01-01,QE11,100.00,94.74,100.00,100.0,100.00,100.00,100.00,100.00
5381,2017-01-01,SI03,100.00,100.00,100.00,100.0,100.00,100.00,100.00,100.00
5380,2017-01-01,BKN16,98.08,94.23,80.00,96.0,100.00,88.00,92.00,84.00
5379,2017-01-01,QW01,93.10,93.10,97.14,100.0,91.43,94.29,97.14,97.14
5378,2017-01-01,QW02,98.51,94.03,93.75,95.0,96.15,93.59,95.00,95.00
...,...,...,...,...,...,...,...,...,...,...
8044,2021-10-01,BKN17,100.00,92.30,NaN,NaN,100.00,100.00,100.00,100.00
7933,2021-10-01,MN01,NaN,NaN,NaN,NaN,96.90,96.90,100.00,100.00
7958,2021-10-01,MN05,NaN,NaN,NaN,NaN,100.00,100.00,100.00,100.00
7959,2021-10-01,MN06,88.20,100.00,100.00,100.0,97.00,96.90,100.00,100.00


In [25]:
#note that the latest date that they collected rating was October, 2021

In [26]:
#rating.to_csv('rating.csv')